# Constants and Libraries

In [5]:
import os, requests, time
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv

if not load_dotenv("./../../../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")

direct_line_secret = os.environ["direct_line_secret"]
sender_id          = "mauromi"
sender_name        = "Mauro Minella"
question           = "Calculate the space routes from Uran to Jupiter" # "my printer is broken"

Environment variables have been loaded ;-)


# Start a new conversation

In [6]:
def new_conversation(direct_line_secret: str):
    # Start a new conversation
    
    headers = {
        'Authorization': f'Bearer {direct_line_secret}',
        'Content-Type': 'application/json'
    }
    start_conv = requests.post(
        'https://directline.botframework.com/v3/directline/conversations',
        headers=headers
    )
    
    conversation_id = start_conv.json()['conversationId']
    return conversation_id


conversation_id = new_conversation(direct_line_secret = direct_line_secret)

conversation_id

'FzuOBlXb1Tj1C9dXSU2jUZ-fr'

# Send a message to the bot

In [7]:
def send_message_to_bot(conversation_id: str, question: str, sender_id: str="user", sender_name: str=""):

    if sender_name=="":
        sender_name = sender_id

    message = {
        'type': 'message',
        'from': {'id': sender_id, 'name': sender_name},
        'text': question
    }

    headers = {
        'Authorization': f'Bearer {direct_line_secret}',
        'Content-Type': 'application/json'
    }
    
    response = requests.post(
        f'https://directline.botframework.com/v3/directline/conversations/{conversation_id}/activities',
        headers=headers,
        json=message,
    )
    return response.json();


send_message_to_bot(conversation_id=conversation_id, question=question, sender_id=sender_id)

{'id': 'FzuOBlXb1Tj1C9dXSU2jUZ-fr|0000000'}

# Poll the activities endpoint

In [8]:
def print_conversation(conversation_id: str, last_question: str):
    activities_url = f"https://directline.botframework.com/v3/directline/conversations/{conversation_id}/activities"
    headers = {
        'Authorization': f'Bearer {direct_line_secret}',
        'Content-Type': 'application/json'
    }
    
    while True:
        response = requests.get(activities_url, headers=headers)
        activities = response.json()['activities']
        if ((activities[-1]["type"] == "message" and activities[-1]["text"] != last_question)
            or (activities[-1]["type"] == "event" and activities[-1]["valueType"] == "DynamicPlanFinished")):
            if (activities[-1]["type"] == "event"):
                print(f"Status: {activities[-1]['valueType']}. FINISHED!\n\n")
            else:
                print("FINISHED!\n\n")
            break
        else:
            time.sleep(1)
            if (activities[-1]["type"] == "event"):
                print(f"Status: {activities[-1]['valueType']}. Waiting for answer to be completed...")
            else:
                print("Waiting for answer to be completed...")

    i = 0

    for activity in activities:
        if activity["type"] == "message":
            i += 1
            display(Markdown(f'Message {i} - {activity["from"]["name"]} says: {activity["text"]}\n'))

    activities = [activity for activity in activities if activity.get('type') == 'message']

    return activities

activities = print_conversation(conversation_id=conversation_id, last_question=question)

Waiting for answer to be completed...
Waiting for answer to be completed...
Waiting for answer to be completed...
Status: DynamicPlanStepBindUpdate. Waiting for answer to be completed...
Status: DynamicPlanStepBindUpdate. Waiting for answer to be completed...
Status: DynamicPlanStepFinished. Waiting for answer to be completed...
Status: DynamicPlanFinished. FINISHED!




Message 1 - mauromi says: Calculate the space routes from Uran to Jupiter


Message 2 - space routes calculator says: The space route from Uranus to Jupiter involves traveling through Saturn. The sequence of planets in the solar system is as follows:

1. Mercury
2. Venus
3. Earth
4. Mars
5. Jupiter
6. Saturn
7. Uranus
8. Neptune

Therefore, the complete route from Uranus to Jupiter is:
Uranus -> Saturn -> Jupiter

Interstellar travel between Uranus and Jupiter requires passing through Saturn. The distance between these planets is significant, and the travel time depends on the speed of the spacecraft used. Space missions must consider the gravity of the planets and their orbits to optimize the route and reduce fuel consumption.


In [9]:
activities

[{'type': 'message',
  'id': 'FzuOBlXb1Tj1C9dXSU2jUZ-fr|0000000',
  'timestamp': '2025-06-27T07:59:08.8882121Z',
  'serviceUrl': 'https://directline.botframework.com/',
  'channelId': 'directline',
  'from': {'id': 'mauromi', 'name': 'mauromi'},
  'conversation': {'id': 'FzuOBlXb1Tj1C9dXSU2jUZ-fr'},
  'text': 'Calculate the space routes from Uran to Jupiter'},
 {'type': 'message',
  'id': 'FzuOBlXb1Tj1C9dXSU2jUZ-fr|0000007',
  'timestamp': '2025-06-27T07:59:15.9620448Z',
  'channelId': 'directline',
  'from': {'id': 'c20bc3d1-1443-a2ee-e236-d53c01154200',
   'name': 'space routes calculator',
   'role': 'bot'},
  'conversation': {'id': 'FzuOBlXb1Tj1C9dXSU2jUZ-fr'},
  'textFormat': 'markdown',
  'membersAdded': [],
  'membersRemoved': [],
  'reactionsAdded': [],
  'reactionsRemoved': [],
  'text': 'The space route from Uranus to Jupiter involves traveling through Saturn. The sequence of planets in the solar system is as follows:\n\n1. Mercury\n2. Venus\n3. Earth\n4. Mars\n5. Jupiter\n6.

# All in one

In [10]:
def InvokeSpaceRoutesCalculator(question: str, conversation_id: str = ""):
    if conversation_id == "":
        conversation_id = new_conversation(direct_line_secret = direct_line_secret)
    send_message_to_bot(conversation_id=conversation_id, question=question, sender_id=sender_id)
    activities = print_conversation(conversation_id=conversation_id, last_question=question)
    return activities

In [11]:
question= "come posso andare su Marte, a partire dalla Terra?" # "I need a plan for my next vacation in Australia"

InvokeSpaceRoutesCalculator(question)

Waiting for answer to be completed...
Waiting for answer to be completed...
Waiting for answer to be completed...
Status: DynamicPlanStepBindUpdate. Waiting for answer to be completed...
Status: DynamicPlanStepFinished. Waiting for answer to be completed...
Status: DynamicPlanStepFinished. Waiting for answer to be completed...
Status: DynamicPlanFinished. FINISHED!




Message 1 - mauromi says: come posso andare su Marte, a partire dalla Terra?


Message 2 - space routes calculator says: Per calcolare la rotta spaziale dal pianeta Terra al pianeta Marte nel sistema solare, possiamo seguire la sequenza dei pianeti. Ecco il percorso completo:

1. Terra
2. Marte

Informazioni dettagliate sui percorsi interstellari:
- La distanza media tra la Terra e Marte è di circa 225 milioni di chilometri.
- Il viaggio spaziale tra la Terra e Marte può richiedere dai 6 ai 9 mesi, a seconda della velocità della navicella spaziale e della posizione relativa dei pianeti.
- Le missioni spaziali devono considerare le finestre di lancio ottimali, che si verificano ogni 26 mesi, quando i pianeti sono allineati favorevolmente.

Se hai bisogno di ulteriori dettagli o informazioni specifiche, fammi sapere!


[{'type': 'message',
  'id': '6zzpZfbz0xBI7xY3eD3L2n-fr|0000000',
  'timestamp': '2025-06-27T07:59:48.585239Z',
  'serviceUrl': 'https://directline.botframework.com/',
  'channelId': 'directline',
  'from': {'id': 'mauromi', 'name': 'mauromi'},
  'conversation': {'id': '6zzpZfbz0xBI7xY3eD3L2n-fr'},
  'text': 'come posso andare su Marte, a partire dalla Terra?'},
 {'type': 'message',
  'id': '6zzpZfbz0xBI7xY3eD3L2n-fr|0000007',
  'timestamp': '2025-06-27T07:59:55.1443349Z',
  'channelId': 'directline',
  'from': {'id': 'c20bc3d1-1443-a2ee-e236-d53c01154200',
   'name': 'space routes calculator',
   'role': 'bot'},
  'conversation': {'id': '6zzpZfbz0xBI7xY3eD3L2n-fr'},
  'textFormat': 'markdown',
  'membersAdded': [],
  'membersRemoved': [],
  'reactionsAdded': [],
  'reactionsRemoved': [],
  'text': 'Per calcolare la rotta spaziale dal pianeta Terra al pianeta Marte nel sistema solare, possiamo seguire la sequenza dei pianeti. Ecco il percorso completo:\n\n1. Terra\n2. Marte\n\nInformaz